# Recording Events
AgentOps has a number of different [Event Types](https://docs.agentops.ai/v1/details/events)

In [ ]:
# Create new session
agentops.start_session()

# Optionally, we can add tags to the session
# agentops.start_session(['Hello Tracker'])

The easiest way to record actions is through the use of AgentOp's decorators

In [0]:
from agentops import record_function

@record_function("add numbers")
def add(x, y):
    return x + y

add(2,4)

We can also manually craft an event exactly the way we want

In [0]:
from agentops import ActionEvent

message = {"role": "user", "content": "Hello"},
response = openai.chat.completions.create(
    model='gpt-3.5-turbo', messages=message, temperature=0.5)

if "hello" in str(response.choices[0].message.content).lower():
    agentops.record(ActionEvent(action_type="Agent says hello", params=str(message), returns=str(response.choices[0].message.content) ))

In [ ]:
agentops.end_session('Success')

## Tool Event
Agents use tools. These tools are useful to track with information such as name, end status, runtime, etc. To record tool usage, you can create and record a `ToolEvent` similar to above.

In [ ]:
from agentops import ToolEvent, record
def scrape_website(url: str):
    tool_event = ToolEvent(name='scrape_website', params={'url':url}) # the start timestamp is set when the obj is created
    result = integration.scrape_website(data) # perform tool logic
    tool_event.returns = result
    record(tool_event)

## Error Events
Error events can be used alone or in reference to another event. Lets add a catch block to the code above

In [ ]:
from agentops import ToolEvent, record, ErrorEvent
def scrape_website(url: str):
    tool_event = ToolEvent(name='scrape_website', params={'url':url}) # the start timestamp is set when the obj is created
    try:
        result = integration.scrape_website(data) # perform tool logic
        tool_event.returns = result
    except Error as e:
        record(ErrorEvent(message=e, trigger_event=tool_event))
    record(tool_event)